In [ ]:
from datetime import date
import random
import time
import yfinance as yf
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from numpy.fft import fft, ifft, fftshift
import numpy as np
from numpy import log, sqrt, exp


from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.mixture import GaussianMixture


from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

import scipy.stats as stats
from scipy.stats import probplot, laplace, norm, t, poisson
from scipy.linalg import solve_banded
from scipy.optimize import minimize, differential_evolution
from scipy.integrate import quad
from scipy.special import roots_laguerre
from scipy.interpolate import interp1d
from scipy.sparse import diags, kron, identity, csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.stats import multivariate_normal, kstest

import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess

import pymc as pm
import arviz as az

#import aesara.tensor as at

from tensorflow import keras
#from tensorflow.keras.utils import plot_model

#import pyswarms as ps

######################################
#from pmdarima import auto_arima
#from diptest import diptest



In [ ]:
class EfficientFrontier:
    def __init__(self, rfr, mu, cov, assets):
        """
        Parameters:
        - mu: expected returns (1D array of length n)
        - cov: covariance matrix (n x n)
        - allow_short: whether to allow short-selling (weights < 0)
        """
        self.rfr = rfr
        self.mu = mu
        self.cov = cov
        self.assets = assets
        self.n = len(mu)
        self.opt_type = None
        self.lam = 2

        self.allow_short = False

###########################################################################################################################
    def tangential_portfolio(self):
        """
        Compute the tangency portfolio (maximum Sharpe ratio portfolio).
        """
        excess_mu = self.mu - self.rfr  # Excess returns over risk-free rate

        def negative_sharpe(w):
            port_return = np.dot(w, self.mu)
            port_vol = np.sqrt(np.dot(w.T, np.dot(self.cov, w)))
            return -((port_return - self.rfr) / port_vol)

        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1},)
        bounds = None if self.allow_short else tuple((0, 1) for _ in range(self.n))

        result = minimize(
            negative_sharpe,
            x0=np.ones(self.n) / self.n,
            method='SLSQP',
            bounds=bounds,
            constraints=constraints
        )

        return result

    def maximize_mean_variance_portfolio(self): #mean-variance optimal portfolio (MVP)
        """
        Compute the mean-variance optimal portfolio:
        Maximize: expected return - lambda * risk (volatility)
        """
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1},)
        bounds = None if self.allow_short else tuple((0, 1) for _ in range(self.n))

        result = minimize(
            fun=lambda w: -np.dot(w, self.mu) + self.lam * 0.5 * np.dot(w.T, np.dot(self.cov, w)),  # -return + λ*risk
            x0=np.ones(self.n) / self.n,
            bounds=bounds,
            constraints=constraints
        )
        return result

    ######################################################################

    def capitalmarket_portfolio(self, target_return):
        """
        Minimize portfolio volatility for a given target return.
        """
        constraints = (
            {'type': 'eq', 'fun': lambda w: self.rfr + np.dot(w, (self.mu - self.rfr)) - target_return}
        )
        bounds = None if self.allow_short else tuple((0, 1) for _ in range(self.n))

        result = minimize(
            lambda w: np.dot(w.T, np.dot(self.cov, w)),
            x0=np.ones(self.n) / self.n,
            bounds=bounds,
            constraints=constraints
        )
        return result

    def minimize_volatility(self, target_return): # Global Minimum Variance (GMV) portfolio
        """
        Minimize portfolio volatility for a given target return.
        """
        constraints = (
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
            {'type': 'eq', 'fun': lambda w: np.dot(w, self.mu) - target_return}
        )
        bounds = None if self.allow_short else tuple((0, 1) for _ in range(self.n))

        result = minimize(
            lambda w: np.dot(w.T, np.dot(self.cov, w)),
            x0=np.ones(self.n) / self.n,
            bounds=bounds,
            constraints=constraints
        )
        return result

    def maximize_return(self, target_risk):
        """
        Maximize portfolio return subject to a target portfolio risk (volatility).
        """
        constraints = (
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights must sum to 1
            {'type': 'eq', 'fun': lambda w: np.dot(w.T, np.dot(self.cov, w)) - target_risk**2}  # volatility == target risk
        )
        bounds = None if self.allow_short else tuple((0, 1) for _ in range(self.n))  # apply bounds if no short-selling

        result = minimize(
            fun=lambda w: -np.dot(w, self.mu),  # maximize return = minimize negative return
            x0=np.ones(self.n) / self.n,
            bounds=bounds,
            constraints=constraints
        )
        return result

###########################################################################################################################

    def compute_frontier(self, opt_type, num_points=100):
        self.opt_type = opt_type
        """
        Compute the efficient frontier for different optimization strategies.
        """
        # Always compute mean-variance optimal portfolio once
        try:
            res = self.tangential_portfolio()
            if res.success:
                weights = res.x
                tpreturns = np.dot(weights, self.mu)
                tprisks = np.sqrt(np.dot(weights.T, np.dot(self.cov, weights)))
                tpweights = weights
        except Exception as e:
            print(f"Tangential PF Optimization failed: {e}")

        try:
            res = self.maximize_mean_variance_portfolio()
            if res.success:
                weights = res.x
                mvpreturns = np.dot(weights, self.mu)
                mvprisks = np.sqrt(np.dot(weights.T, np.dot(self.cov, weights)))
                mvpweights = weights
        except Exception as e:
            print(f"Mean-Variance Optimization failed: {e}")


        risks, returns, weights = [], [], []
        risks1, returns1, weights1 = [], [], []

        # Create a range of target returns or risks
        min_return, max_return = 0.0 , np.max(self.mu)
        min_risk, max_risk = 0.0 , np.max(np.sqrt(np.diag(self.cov)))

        target_returns = np.linspace(min_return, max_return, num_points)
        target_risks = np.linspace(min_risk, max_risk, num_points)

        # Compute rest based on user-specified type
        for i in range(num_points):
            try:
                if self.opt_type == 'minvar':
                    res = self.minimize_volatility(target_returns[i])
                    if res.success:
                        weight = res.x
                        returns.append(np.dot(weight, self.mu))
                        risks.append(np.sqrt(np.dot(weight.T, np.dot(self.cov, weight))))
                        weights.append(weight)
                elif self.opt_type == 'maxret':
                    res = self.maximize_return(target_risks[i])
                    if res.success:
                        weight = res.x
                        returns.append(np.dot(weight, self.mu))
                        risks.append(np.sqrt(np.dot(weight.T, np.dot(self.cov, weight))))
                        weights.append(weight)
                elif self.opt_type == 'cml':
                    res = self.minimize_volatility(target_returns[i])
                    if res.success:
                        weight = res.x
                        returns1.append(np.dot(weight, self.mu))
                        risks1.append(np.sqrt(np.dot(weight.T, np.dot(self.cov, weight))))
                        weights1.append(weight)

                    res = self.capitalmarket_portfolio(target_returns[i])
                    if res.success:
                        weight = res.x
                        returns.append(self.rfr + np.dot(weight, (self.mu - self.rfr)))
                        risks.append(np.sqrt(np.dot(weight.T, np.dot(self.cov, weight))))
                        weights.append(weight)
                else:
                    raise ValueError("Unknown optimization type: " + self.opt_type)

            except Exception as e:
                print(f"Optimization failed for point {i}: {e}")
                continue

        print("Efficient Frontier Computed.")

        if self.opt_type == 'cml':

            tolerance = 0.01
            weight = next((w for w in weights if abs(np.sum(w) - 1) < tolerance), None)

            print ('from 100% risky portfolio on CML', weight)

            '''
            try:
                res = self.capitalmarket_portfolio(tpreturns)
                if res.success:
                    weight = res.x
            except Exception as e:
                print(f"Tangential PF Optimization failed: {e}")

            print ('from maximizing sharpe ratio (adjusted return) of risky assests - excess return / risk', weight)
            print ('sum =', np.cumsum(res.x))
            '''

            self.plot_frontier(np.array(returns), np.array(risks), tpreturns, tprisks, weight, np.array(returns1), np.array(risks1))
        else:
            self.plot_frontier(np.array(returns), np.array(risks), mvpreturns, mvprisks, mvpweights)

####################################################################################################################################################

    def plot_frontier(self, returns, risks, p_return, p_risk, p_weights,  returns1 = None, risks1 = None):
        """
        Plot the efficient frontier with assets, MVP point, and portfolio weights (if given).
        """

        fig, ax = plt.subplots(1, 2, figsize=(16, 6), gridspec_kw={'width_ratios': [2.5, 1]})
        asset_risks = np.sqrt(np.diag(self.cov))

        # ----------- Left Plot: Efficient Frontier ----------- #
        #Efficient frontier
        if risks1 is not None:
            sharpe_ratio = (p_return - self.rfr) / p_risk
            ax[0].plot(risks1, returns1, 'b--', lw=2, label='Efficient Frontier of risky assets')
            ax[0].plot(risks, returns, 'k-', lw=2, label=' Capital Market Line : EF for risky + riks free portfolio')
            ax[0].scatter(p_risk, p_return, color='green', marker='o', s=120,
                          label=f'Tangential Portfolio (Sharpe = {sharpe_ratio:.2f})')
            ax[0].annotate('TP', (p_risk, p_return), textcoords="offset points", xytext=(10,10), ha='left', fontsize=10, color='green')
            ax[0].axvline(x=p_risk, color='gray', linestyle='--', linewidth=1, alpha=1)
            ax[0].axhline(y=p_return, color='gray', linestyle='--', linewidth=1, alpha=1)
        else:
            ax[0].plot(risks, returns, 'b-', lw=2, label='Efficient Frontier of risky assets')
            ax[0].scatter(p_risk, p_return, color='green', marker='o', s=120, label='MVP (Max Mean-Variance)')
            ax[0].annotate('MVP', (p_risk, p_return), textcoords="offset points", xytext=(10,10), ha='left', fontsize=10, color='green')

        # Assets
        ax[0].scatter([0], self.rfr, color='black', marker='x', s=100, label='Risk-free asset')
        ax[0].scatter(asset_risks, self.mu, color='red', marker='x', s=100, label='Assets')
        for i, asset in enumerate(self.assets):
            #ax[0].annotate(asset, (asset_risks[i], self.mu[i]), textcoords="offset points", xytext=(0,10), ha='center')
            ax[0].axvline(x=asset_risks[i], color='gray', linestyle='--', linewidth=1, alpha=0.4)
            ax[0].axhline(y=self.mu[i], color='gray', linestyle='--', linewidth=1, alpha=0.4)


        ax[0].set_xlabel('Annualized Risk (Std Dev)')
        ax[0].set_ylabel('Annualized Return')
        ax[0].set_title('Efficient Frontier')
        ax[0].set_xlim([0, max(np.sqrt(np.diag(self.cov)))*1.2])
        ax[0].set_ylim([0, max(self.mu)*1.2])
        #ax[0].grid(True)
        ax[0].legend()

        # ----------- Right Plot: Weights (if provided) ----------- #
        weights_cumsum = np.cumsum(p_weights)

        ax[1].barh(self.assets, p_weights, color='green', alpha=0.7)
        ax[1].set_title("Portfolio Weights at tangential Portfolio or MVP")
        ax[1].set_xlim([0, max(p_weights) * 1.2])
        ax[1].set_xlabel("Weight")

        # Annotate cumulative sum next to each bar
        for i, (weight, cum_weight) in enumerate(zip(p_weights, weights_cumsum)):
            ax[1].text(weight + 0.01, i, f"Cum: {cum_weight:.2f}", va='center', fontsize=9, color='black')

        plt.tight_layout()
        plt.show()

    def plot_oppurtunity_set(self):
        """
        Plot the opportunity set for portfolios with and without short selling.
        """
        fig, axs = plt.subplots(2, 1, figsize=(10, 10), sharey=True)
        titles = ['Without Short Selling', 'With Short Selling']

        all_risks = []
        all_returns = []

        for idx, allow_short in enumerate([False, True]):
            returns, risks = [], []
            npf = 10000

            for _ in range(npf):
                if allow_short:
                    weights = np.random.randn(self.n)
                    weights /= np.sum(weights)
                else:
                    weights = np.random.rand(self.n)
                    weights /= np.sum(weights)

                ret = np.dot(weights, self.mu)
                risk = np.sqrt(np.dot(weights.T, np.dot(self.cov, weights)))

                returns.append(ret)
                risks.append(risk)

            returns = np.array(returns)
            risks = np.array(risks)
            sharpe_ratios = (returns) / risks

            all_returns.append(returns)
            all_risks.append(risks)

            ax = axs[idx]
            sc = ax.scatter(risks, returns,
                            c=sharpe_ratios, cmap='viridis', marker='o', alpha=0.5,
                            label='Random Portfolios')

            # Plot assets
            asset_risks = np.sqrt(np.diag(self.cov))
            ax.scatter(asset_risks, self.mu, color='red', marker='x', s=100, label='Assets')

            # Asset annotations
            for i in range(self.n):
                ax.axvline(x=asset_risks[i], color='gray', linestyle='--', linewidth=1, alpha=0.4)
                ax.axhline(y=self.mu[i], color='gray', linestyle='--', linewidth=1, alpha=0.4)
                ax.text(asset_risks[i], self.mu[i], f'Asset {i}', fontsize=9, ha='right', va='bottom')

            ax.set_title(titles[idx])
            ax.set_xlabel("Annualized Risk (Volatility)")
            if idx == 0:
                ax.set_ylabel("Annualized Return")
            ax.legend()

        # Set same limits across both plots
        for ax in axs:
            ax.set_xlim([0, max(np.sqrt(np.diag(self.cov)))*1.2])
            ax.set_ylim([0, max(self.mu)*1.2])

        # Shared colorbar
        cbar = fig.colorbar(sc, ax=axs[:], shrink=0.9, label='Sharpe Ratio')

        plt.suptitle("Opportunity Set: With vs Without Short Selling")
        #plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.show()

